# ETL Transform

This notebook imports the core dataset so you can begin transformation steps independently of geo-specific work.

## Overview
- Load the dataset from `../data/archive.zip`.
- Compute per-site (State, County, City, Address) daily reading counts.
- Check whether daily counts are constant per site.

In [1]:
# import necessary libraries (single import cell)
import pandas as pd
from pathlib import Path
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load dataset (reusing the same path used in etl_extract_cood.ipynb)
# - Reads compressed CSV from ../data/archive.zip
# - Stores in df; prints shape and previews head

data_path = Path('../data/archive.zip')
df = pd.read_csv(data_path, compression='zip')
print(f'Loaded df with {len(df):,} rows and {df.shape[1]} columns')
df.head()

Loaded df with 1,746,661 rows and 29 columns


,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.2,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.2,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.6,23,NaN


## Check for variance in count of daily readings per site.

In [3]:
# Group by State, County, City, Address, Date Local: are per-site daily counts constant?
import pandas as pd
from IPython.display import display

required = ['State','County','City','Address','Date Local']
missing = [c for c in required if c not in df.columns]
if missing:
    raise KeyError(f"Missing required columns: {missing}")

# Parse Date Local to day-level
_dt = pd.to_datetime(df['Date Local'], errors='coerce')
if not _dt.notna().any():
    raise RuntimeError("Could not parse any dates from 'Date Local'.")

work = df.copy()
work['_date'] = _dt.dt.date
site_key = ['State','County','City','Address']
full_key = site_key + ['_date']

# Count readings per site-day
counts = (
    work.groupby(full_key, dropna=False)
        .size()
        .reset_index(name='n_readings')
)
print(f"Computed counts for {len(counts):,} site-date groups across {counts[site_key].drop_duplicates().shape[0]:,} sites.")

# For each site (State, County, City, Address), check if daily counts are constant across days
per_site_unique = counts.groupby(site_key)['n_readings'].nunique()
all_sites_constant = (per_site_unique == 1).all()
print(f"All sites have the same number of readings per day? {all_sites_constant}")

if not all_sites_constant:
    varying = int((per_site_unique > 1).sum())
    total_sites = int(per_site_unique.size)
    print(f"Sites with varying daily counts: {varying} of {total_sites}")

Computed counts for 412,856 site-date groups across 204 sites.
All sites have the same number of readings per day? False
Sites with varying daily counts: 154 of 204


There is clearly a variance in number of readings per day per

This however, may not be an issue. Finn has discovered a consensus that we use the worst reading.

Daniel is Skeptical about faulty readings being aggregated and suggests the median value for simple aggregation, or to automate outlier detection/removal prior to worse reading aggregate if practical and desirable.

## Data Cleaning

This section profiles the dataset and highlights practical cleanup steps:
- Schema and dtypes overview.
- Missing values by column.
- Exact-duplicate rows and potential duplicate keys (by site + day).
- String hygiene (leading/trailing whitespace) and standardization.
- Date parsing and coverage for `Date Local`.
- Numeric sanity checks (negatives, basic outlier counts).

The `clean_df` function applies safe, non-destructive fixes (trim strings, parse dates, drop exact duplicate rows).

## AQI Columns (Optional)

AQI fields (e.g., `CO AQI`, `SO2 AQI`) are derived and sparse because they only apply on rows for that pollutant and when averaging rules are met. If you don’t need them downstream, you can drop them to reduce size and avoid confusion. The next cell creates `df_no_aqi` without AQI columns and reports memory savings.

In [4]:
# Drop AQI columns to create df_no_aqi (optional)
from IPython.display import display

aqi_cols = [c for c in df.columns if c.endswith(' AQI') or c == 'AQI']
print(f"AQI columns detected: {aqi_cols}" if aqi_cols else "No AQI columns detected.")

def _mem_bytes(frame):
    try:
        return int(frame.memory_usage(deep=True).sum())
    except Exception:
        return 0

mem_before = _mem_bytes(df)
if aqi_cols:
    df_no_aqi = df.drop(columns=aqi_cols)
else:
    df_no_aqi = df.copy()
mem_after = _mem_bytes(df_no_aqi)

print(f"Shape before: {df.shape} -> after: {df_no_aqi.shape}")
if mem_before and mem_after:
    saved = (mem_before - mem_after) / 1e6
    print(f"Approx memory before: {mem_before/1e6:.1f} MB; after: {mem_after/1e6:.1f} MB; saved: {saved:.1f} MB")

# Peek columns removed
if aqi_cols:
    print("Dropped columns:")
    display(aqi_cols)

AQI columns detected: ['NO2 AQI', 'O3 AQI', 'SO2 AQI', 'CO AQI']
Shape before: (1746661, 29) -> after: (1746661, 25)
Approx memory before: 1280.7 MB; after: 1224.8 MB; saved: 55.9 MB
Dropped columns:
Shape before: (1746661, 29) -> after: (1746661, 25)
Approx memory before: 1280.7 MB; after: 1224.8 MB; saved: 55.9 MB
Dropped columns:


['NO2 AQI', 'O3 AQI', 'SO2 AQI', 'CO AQI']

## Zero Values: Hours vs Measurements

- 0 in columns ending with `Hour` (e.g., `CO 1st Max Hour`) indicates the midnight hour (range 0–23). This is an index, not a concentration.
- 0 in measurement columns (e.g., `SO2 Mean`, `CO 1st Max Value`) can be legitimate when values are at/under method detection limit or due to rounding. Bulk-converting 0 → NaN is not recommended without domain qualifiers.
- Below we verify hour columns are within [0,23], then summarize zero rates for measurement-like fields and show near-zero distributions.

In [5]:
# Validate hour-type columns (should be integers in [0, 23])
import pandas as pd
from IPython.display import display

hour_cols = [c for c in df.columns if isinstance(c, str) and c.endswith(' Hour')]
if not hour_cols:
    print("No columns ending with ' Hour' found.")
else:
    print(f"Hour columns: {hour_cols}")
    rows = []
    for c in hour_cols:
        s = pd.to_numeric(df[c], errors='coerce')
        n = int(s.notna().sum())
        out_of_range = int(((s < 0) | (s > 23)).sum())
        non_integer = int((s.notna() & (s % 1 != 0)).sum())
        min_v = float(s.min()) if n else float('nan')
        max_v = float(s.max()) if n else float('nan')
        rows.append({
            'column': c,
            'non_null': n,
            'min': min_v,
            'max': max_v,
            'non_integer': non_integer,
            'out_of_[0,23]': out_of_range,
            'zeros': int((s == 0).sum()),
        })
    hour_summary = pd.DataFrame(rows).sort_values('out_of_[0,23]', ascending=False)
    display(hour_summary)

Hour columns: ['NO2 1st Max Hour', 'O3 1st Max Hour', 'SO2 1st Max Hour', 'CO 1st Max Hour']


,column,non_null,min,max,non_integer,"out_of_[0,23]",zeros
0,NO2 1st Max Hour,1746661,0.0,23.0,0,0,168031
1,O3 1st Max Hour,1746661,0.0,23.0,0,0,88678
2,SO2 1st Max Hour,1746661,0.0,23.0,0,0,158831
3,CO 1st Max Hour,1746661,0.0,23.0,0,0,544131


In [6]:
# Zero analysis for measurement-like columns ('* Mean', '* 1st Max Value')
import pandas as pd
import numpy as np
from IPython.display import display

# Identify measurement-like columns (wide layout)
measure_cols = [
    c for c in df.columns
    if isinstance(c, str) and (c.endswith(' Mean') or c.endswith(' 1st Max Value'))
]

if not measure_cols:
    print("No measurement-like columns ('* Mean' or '* 1st Max Value') found.")
else:
    rows = []
    for c in measure_cols:
        s = pd.to_numeric(df[c], errors='coerce')
        non_null = int(s.notna().sum())
        zeros = int((s == 0).sum())
        zero_rate = zeros / non_null if non_null else np.nan
        rows.append({
            'column': c,
            'non_null': non_null,
            'zeros': zeros,
            'zero_rate': zero_rate,
        })
    zero_summary = pd.DataFrame(rows).sort_values(['zero_rate','zeros'], ascending=False)
    print("Top zero rates for measurement-like columns:")
    display(zero_summary.head(10))

Top zero rates for measurement-like columns:


,column,non_null,zeros,zero_rate
5,SO2 1st Max Value,1746661,153488,0.087875
4,SO2 Mean,1746661,146207,0.083707
7,CO 1st Max Value,1746661,58122,0.033276
6,CO Mean,1746661,57672,0.033018
1,NO2 1st Max Value,1746661,8276,0.004738
0,NO2 Mean,1746661,8244,0.004720
2,O3 Mean,1746661,600,0.000344
3,O3 1st Max Value,1746661,600,0.000344
